In [19]:
# Import packages
import pandas as pd
import numpy as np
import requests

In [20]:
# API
title = ['France_cpi' , 'Germany_cpi' , 'Belgium_cpi' , 'Italy_cpi' , 'Poland_cpi' , 'Spain_cpi' , 'The Netherlands_cpi' , 'Ireland_cpi' , 'Austria_cpi']
series_codes = ['CPALTT01FRM657N' , 'CPALTT01DEM657N' , 'CPALTT01BEM657N' , 'CPALTT01ITM657N' , 'CPALTT01PLM657N' , 'CPALTT01ESM657N' , 'CPALTT01NLM657N' , 'CPALTT01IEM657N' , 'CPALTT01ATM657N']

In [27]:
USD_EUR_exrate = requests.get('https://api.stlouisfed.org/fred/series/observations?series_id=DEXUSEU&api_key=ec84959f0124fa43e3b547e8cf788a1a&file_type=json').json()

In [28]:
# Extracting necessary data from API

# Get observations from JSON data store as a pandas dataframe
USD_EUR_exrate_df = pd.DataFrame(USD_EUR_exrate['observations'])
# Change 'value' column containing the USD/EUR exchange rate from a string to a numeric
USD_EUR_exrate_df['value'] = pd.to_numeric(USD_EUR_exrate_df['value'], errors='coerce')
# Convert daily exchange rate data to monthly data
# Remove the day part of the month string
USD_EUR_exrate_df['date'] = USD_EUR_exrate_df['date'].str[:-3]
# Aggregate to monthly by taking the month mean exchange rate
USD_EUR_exrate_df = USD_EUR_exrate_df.groupby(by='date').mean('value').reset_index()
# Change 'date' column from a string to a date
USD_EUR_exrate_df['date'] = pd.to_datetime(USD_EUR_exrate_df['date'], errors='coerce')
# Get the exchange rate in Euros per Dollar instead of Dollars per Euro
USD_EUR_exrate_df['EUR_USD_exrate'] = 1 / USD_EUR_exrate_df['value']
# Subset to relavent columns
EUR_USD_exrate_df = USD_EUR_exrate_df[['date' , 'EUR_USD_exrate']]
EUR_USD_exrate_df

,date,EUR_USD_exrate
0,1999-01-01,0.862742
1,1999-02-01,0.892635
2,1999-03-01,0.918622
3,1999-04-01,0.934536
4,1999-05-01,0.940729
...,...,...
306,2024-07-01,0.921949
307,2024-08-01,0.907426
308,2024-09-01,0.900556
309,2024-10-01,0.917894


In [36]:
# Call the FRED APIs in loop to get series for 9 Euro using economies. Merge series together with the EUR/USD exchange rate data
exrate_cpis_df = EUR_USD_exrate_df
for i in range(len(series_codes)):
    series = series_codes[i]
    fred_url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key=ec84959f0124fa43e3b547e8cf788a1a&file_type=json'
    fred_json = requests.get(fred_url).json()
    fred_df = pd.DataFrame(fred_json['observations'])
    fred_df[str(title[i])] = pd.to_numeric(fred_df['value'], errors='coerce')
    fred_df['date'] = pd.to_datetime(fred_df['date'], errors='coerce')
    fred_df = fred_df[['date' , str(title[i])]]
    exrate_cpis_df = exrate_cpis_df.merge(fred_df , on='date' , how='outer')


In [37]:
# Subset dataframe to only show dates after the Euro ws implemented and before the CPI data ends in February 2024
exrate_cpis_df_sub = exrate_cpis_df[(exrate_cpis_df['date'] >= '1999-01-01') & (exrate_cpis_df['date'] <= '2024-02-01')].reset_index(drop=True)

In [38]:
# Calculate moving average with pandas' rolling method
for i in range(len(title)):
    exrate_cpis_df_sub[f'{title[i]}_ma'] = exrate_cpis_df_sub[f'{title[i]}'].rolling(window=12).mean()

In [39]:
exrate_cpis_df_sub

,date,EUR_USD_exrate,France_cpi,Germany_cpi,Belgium_cpi,Italy_cpi,Poland_cpi,Spain_cpi,The Netherlands_cpi,Ireland_cpi,Austria_cpi,France_cpi_ma,Germany_cpi_ma,Belgium_cpi_ma,Italy_cpi_ma,Poland_cpi_ma,Spain_cpi_ma,The Netherlands_cpi_ma,Ireland_cpi_ma,Austria_cpi_ma
0,1999-01-01,0.862742,-0.281366,-0.119048,0.342700,0.091912,1.5,0.367420,0.055394,-0.689655,-0.082057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1999-02-01,0.892635,0.307811,0.119190,0.218579,0.183655,0.6,0.059148,0.498270,0.520833,0.191624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-03-01,0.918622,0.332438,0.000000,0.081788,0.183318,1.0,0.440947,0.867649,0.345423,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1999-04-01,0.934536,0.280362,0.476190,0.381367,0.274474,0.8,0.369820,0.218460,0.516351,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1999-05-01,0.940729,0.012708,0.000000,0.176391,0.182482,0.7,-0.003170,0.027248,0.342466,0.191256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2023-10-01,0.946560,0.143751,0.000000,0.343133,-0.166251,0.3,0.289374,0.448254,0.300000,0.329489,0.326581,0.311084,0.031234,0.140353,0.500,0.285012,-0.027394,0.411297,0.436659
298,2023-11-01,0.924274,-0.168876,-0.424448,0.170980,-0.499584,0.7,-0.348358,-1.041259,-0.797607,0.246305,0.286169,0.261029,0.064331,0.056384,0.500,0.265994,0.139245,0.318652,0.435558
299,2023-12-01,0.916641,0.135329,0.085251,0.434479,0.167364,0.1,0.024718,0.039557,0.502513,0.409500,0.304740,0.304779,0.113567,0.049270,0.500,0.255523,0.103003,0.377925,0.455303
300,2024-01-01,0.917523,-0.168933,0.170358,0.486674,0.334169,0.4,0.084725,0.521945,-1.300000,-0.081566,0.258544,0.237998,0.146294,0.070114,0.325,0.280100,0.261887,0.339327,0.376729


In [58]:
var_list = ['date' , 'EUR_USD_exrate' , 'France_cpi_ma' , 'Germany_cpi_ma' , 'Belgium_cpi_ma' , 'Italy_cpi_ma' , 'Poland_cpi_ma' , 'Spain_cpi_ma' , 'The Netherlands_cpi_ma' , 'Ireland_cpi_ma' , 'Austria_cpi_ma']

In [67]:
exrate_cpis_df_sub = exrate_cpis_df_sub[var_list][exrate_cpis_df_sub['date'] >= '2020-01-01']

In [68]:
# Transform from wide to long format
exrate_cpis_df_long = exrate_cpis_df_sub.melt(id_vars='date')

In [69]:
exrate_cpis_df_long.to_csv('/Users/sambickel-barlow/Desktop/PP434/Portfolio/CC6/exrate_cpis.csv')